# 🚀 YOLOv8 Inference API – FastAPI Server
This notebook builds a FastAPI app to serve the YOLOv8 defect detection model.

## 📦 Step 1: Install dependencies

In [ ]:
!pip install ultralytics fastapi uvicorn python-multipart opencv-python aiofiles --quiet

## 🧠 Step 2: Load YOLOv8 model and test locally

In [ ]:

from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import io

# Load trained model
model = YOLO('runs/detect/yolov8-defect/weights/best.pt')  # adjust path if needed


## 🌐 Step 3: Define FastAPI app for image upload and inference

In [ ]:

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import shutil
import os

app = FastAPI()

@app.post("/predict")
async def predict_image(file: UploadFile = File(...)):
    contents = await file.read()
    nparr = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    results = model(img)
    detections = results[0].boxes

    output = []
    if detections is not None:
        for box in detections:
            xyxy = box.xyxy.cpu().numpy()[0].tolist()
            cls_id = int(box.cls.cpu().numpy()[0])
            conf = float(box.conf.cpu().numpy()[0])
            label = model.names[cls_id]
            output.append({
                "class": label,
                "confidence": round(conf, 3),
                "bbox": [round(x, 2) for x in xyxy]
            })

    return JSONResponse(content={"detections": output})


## ▶️ Step 4: Run the API locally

In [ ]:

# Run in terminal or script, not inside notebook
# uvicorn api.main:app --host 0.0.0.0 --port 8000
